In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
if not os.getcwd().endswith('CIoTS'):
    os.chdir('../..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from CIoTS import *

### Helper: check if data exists

In [12]:
import os.path

def check_setups(setups, data_path):
    return not missing_setups(setups, data_path)

def missing_setups(setups, data_path):
    missing = []
    for dim, in_edges, tau, autocorr, _, run in setups:
        if not os.path.isfile(data_path + f't={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle'):
            missing.append((dim, in_edges, tau, autocorr, run))
    return missing

# Execution

In [13]:
from itertools import product

dimensions = [3,5,10]
incoming_edges = [2,3,4]
taus = [5,10,15,20]
autocorrs = [False, True]
data_length = [1000]
runs = range(10)


setups = list(product(dimensions, incoming_edges, taus, autocorrs, data_length, runs))

## 3. Evaluate each iteration

In [ ]:
import pickle

data_path = 'notebooks/ICML/nonstationary_data/'
results_path = 'notebooks/ICML/icml_results_nonstationary/'
results = pd.read_csv(results_path + f'experiment3.csv')

algorithms = [(pc_incremental, 'PC incremental'),
              (pc_incremental_extensive, 'PC extensive'),
              (pc_incremental_pc1, 'PC1 incremental'),
              (pc_incremental_pc1mci, 'PCMCI incremental')]

if not check_setups(setups, data_path):
    print('Mising setups:')
    print(missing_setups(setups, data_path))   
    print(len(missing_setups(setups, data_path)))
    
for dim, in_edges, tau, autocorr, _, run in setups[82:]:
    generator = pickle.load(open(data_path + f't={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle', 'rb'))
    max_tau = 2 * tau
    
    df_dict = {'dimension': [dim]*max_tau, 'max time lag': [tau]*max_tau, 'incoming edges': [in_edges]*max_tau, 
               'run': [run]*max_tau, 'autocorr': [autocorr]*max_tau, 'tau estimate': list(range(1, max_tau+1))}
    
    algo_graphs = {}
    for algorithm, name in algorithms:
        _, graphs, _, stopper, _ = algorithm(tigramite_partial_corr_test, generator.ts, max_p=max_tau, 
                                             use_stopper=False, alpha=0.01, verbose=True)

        algo_graphs[name] = graphs
        confusion, confusion_delta = evaluate_edge_deletion(generator.graph,
                                                            [{'graph': graphs[t], 'p_iter': t} 
                                                             for t in range(1, max_tau+1)],
                                                            dim)
        added_edges = (confusion_delta['tp'] + confusion_delta['fp']).tolist()
        f1_scores = [evaluate_edges(generator.graph, graphs[t])['f1-score'] for t in range(1, max_tau+1)]
        bics = [stopper.scores()[t] for t in range(1, max_tau+1)]
        
        eval_dict = {name + '_tn': confusion['tn'].tolist(), name + '_fp': confusion['fp'].tolist(),
                     name + '_tp': confusion['tp'].tolist(), name + '_fn': confusion['fn'].tolist(),
                     name + '_f1': f1_scores, name + '_bics': bics, name + '_added_edges': added_edges}
        df_dict.update(eval_dict)
        
    pickle.dump(algo_graphs, open(results_path + 'exp3_graphs/' + 
                                  f'graphs-t={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle', 'wb'))
    df = pd.DataFrame(df_dict)
    results = results.append(df, ignore_index=True)

    results.to_csv(results_path + f'experiment3.csv', index=False)

/home/Daniel.Thevessen/Causality/CIoTS/src/tigramite/tigramite/independence_tests.py:1144: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  beta_hat = numpy.linalg.lstsq(z, y)[0]


## 4. Evaluate VAR models

In [14]:
import pickle


def graph2Var(graph, dim, tau, mapping):
    inverted_mapping = {v: k for k, v in mapping.items()}
    params = np.zeros((dim * tau, dim))

    for x_t in range(dim):
        input_nodes = list(graph.predecessors(mapping[x_t]))
        inputs = np.array([inverted_mapping[x] for x in input_nodes])
        for i in inputs:
            params[i - dim, x_t] = graph.edges[(mapping[i], mapping[x_t])]['weight']
    return params

data_path = 'notebooks/ICML/nonstationary_data/'
results_path = 'notebooks/ICML/icml_results_nonstationary/'
results = pd.DataFrame()

names = ['PC1 incremental', 'PC extensive', 'PC incremental', 'PCMCI incremental']

for dim, in_edges, tau, autocorr, _, run in setups:
    generator = pickle.load(open(data_path + f't={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle', 'rb'))
    algo_graphs = pickle.load(open(results_path + 'exp3_graphs/' + 
                                   f'graphs-t={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle', 'rb'))
    max_tau = 2*tau
    df_dict = {'dimension': [dim]*max_tau, 'max time lag': [tau]*max_tau, 'incoming edges': [in_edges]*max_tau, 
               'run': [run]*max_tau, 'autocorr': [autocorr]*max_tau, 'tau estimate': list(range(1, max_tau+1))}
    
    true_mapping, true_matrix = transform_ts(generator.ts, tau)
    true_params = graph2Var(generator.graph, dim, tau, true_mapping)
    
    # each algorithm
    for name in names:
        df_dict[name] = []
        graphs = algo_graphs[name]
        
        assert len(graphs) == max_tau
        for est_tau, graph in graphs.items():
            model = VAR(est_tau)
            node_mapping, data_matrix = transform_ts(generator.ts, est_tau)
            model.fit_from_graph(dim, data_matrix, graph, node_mapping)
            est_params = model.params[1:]
            df_dict[name].append(evaluate_parameters(true_params, est_params))
    
    # VAR for different tau'
    df_dict['complete VAR'] = []
    for est_tau in range(1, max_tau+1):
        model = VAR(est_tau)
        model.fit(generator.ts)
        est_params = model.params[1:]
        df_dict['complete VAR'].append(evaluate_parameters(true_params, est_params))
    
    # True Graph VAR
    model = VAR(tau)
    model.fit_from_graph(dim, true_matrix, generator.graph, true_mapping)
    est_params = model.params[1:]
    df_dict['true Graph'] = [evaluate_parameters(true_params, est_params)] * max_tau
    
    df = pd.DataFrame(df_dict)
    results = results.append(df)
    
    results.to_csv(results_path + f'experiment4.csv', index=False)


/home/Daniel.Thevessen/Causality/CIoTS/CIoTS/evaluation.py:123: RuntimeWarning: Mean of empty slice.
  mse_fp = se[(true_params == 0) & (est_params != 0)].mean()
/home/Daniel.Thevessen/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## 5. Incremental vs Non-Incremental

In [15]:
import pickle

def graph2Var(graph, dim, tau, mapping):
    inverted_mapping = {v: k for k, v in mapping.items()}
    params = np.zeros((dim * tau, dim))

    for x_t in range(dim):
        input_nodes = list(graph.predecessors(mapping[x_t]))
        inputs = np.array([inverted_mapping[x] for x in input_nodes])
        for i in inputs:
            params[i - dim, x_t] = graph.edges[(mapping[i], mapping[x_t])]['weight']
    return params

data_path = 'notebooks/ICML/nonstationary_data/'
results_path = 'notebooks/ICML/icml_results_nonstationary/'
results = pd.DataFrame()

k = 2

if not check_setups(setups, data_path):
    print('Mising setups:')
    print(missing_setups(setups, data_path))

for dim, in_edges, tau, autocorr, _, run in setups:
    generator = pickle.load(open(data_path + f't={tau}_d={dim}_in={in_edges}_autocorr={autocorr}_{run}.pickle', 'rb'))
    
    true_mapping, true_matrix = transform_ts(generator.ts, tau)
    true_params = graph2Var(generator.graph, dim, tau, true_mapping)
    
    df_dict = {'dimension': dim, 'max time lag': tau, 'incoming edges': in_edges, 'run': run, 'autocorr': autocorr}
    
    # incremental
    stopper = ICStopper(dim=dim, patiency=2)
    graph = pc_incremental_pc1(tigramite_partial_corr_test, ts=generator.ts, max_p=2*tau, stopper=stopper)
    eval_results = evaluate_edges(generator.graph, graph)
    for name, score in eval_results.items():
        df_dict[f'PC1 incremental - {name}'] = score
    mapping, matrix = transform_ts(generator.ts, stopper.best_tau)
    model = VAR(stopper.best_tau)
    model.fit_from_graph(dim, matrix, graph, mapping)
    est_params = model.params[1:]
    mses = evaluate_parameters(true_params, est_params)
    df_dict.update({'PC1 incremental - MSE full': mses[0],
                    'PC1 incremental - MSE TR': mses[1],
                    'PC1 incremental - MSE FP': mses[2]})
    
    # non-incremental
    for offset in range(-k, k+1):
        offset_str = f'{offset:+}' if offset != 0 else ''
        graph = pc_incremental_pc1(tigramite_partial_corr_test, ts=generator.ts, 
                                   start=0, step=tau+offset, max_p=tau+offset)
        eval_results = evaluate_edges(generator.graph, graph)
        for name, score in eval_results.items():
            df_dict[f'PC1 tau{offset_str} - {name}'] = score
        mapping, matrix = transform_ts(generator.ts, tau+offset)
        model = VAR(tau+offset)
        model.fit_from_graph(dim, matrix, graph, mapping)
        est_params = model.params[1:]
        mses = evaluate_parameters(true_params, est_params)
        df_dict.update({f'PC1 tau{offset_str} - MSE full': mses[0],
                        f'PC1 tau{offset_str} - MSE TR': mses[1],
                        f'PC1 tau{offset_str} - MSE FP': mses[2]})
    
    results = results.append(df_dict, ignore_index=True)
    results.to_csv(results_path + f'experiment5.csv', index=False)

/home/Daniel.Thevessen/Causality/CIoTS/src/tigramite/tigramite/independence_tests.py:1144: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  beta_hat = numpy.linalg.lstsq(z, y)[0]
/home/Daniel.Thevessen/Causality/CIoTS/CIoTS/evaluation.py:123: RuntimeWarning: Mean of empty slice.
  mse_fp = se[(true_params == 0) & (est_params != 0)].mean()
/home/Daniel.Thevessen/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/site-packages/scipy/stats/stats.py:3010: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


# Visualization

## 3. Visualize iterations

In [ ]:
from math import floor

visualize = [(20, 3, 4, False)]
names = ['PC1 incremental', 'PC extensive', 'PC incremental']
prop = 'bics'

width = 0.4
results_path = 'notebooks/ICML/icml_results_v2/'

results = pd.read_csv(results_path + 'experiment3.csv')
true_taus = np.unique(results['max time lag'])

for group, result in results.groupby(['max time lag', 'dimension', 'incoming edges', 'autocorr', 'run']): 
    
    tau = int(group[0])
    dim = int(group[1])
    in_edges = int(group[2])
    autocorr = float(group[3])
    run = int(group[4])
    
    if (tau, dim, in_edges, autocorr) not in visualize:
        continue
    
    colors = plt.cm.CMRmap(np.linspace(0,1,len(names)+2))
    plt.figure(dpi=200, figsize=(15, 8))
    
    x = result['tau estimate']
    for i, name in enumerate(names):
        y = result[name + '_' + prop]
        plt.plot(x, y, color=colors[i+1], label=name)
    
    plt.xlabel('iteration $\\tau$')
    plt.ylabel(prop)
    plt.title(f'{prop} for dimensionality={dim}, incoming edges={in_edges}, autocorr={autocorr}, run={run}')
    
    plt.legend()
    plt.show()